In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "../data/route25_potential_and_real_stops.csv"

In [3]:
r25 = pd.read_csv(path)

In [21]:
r25.head()

,UniqueNum,Distance from initial stop (feet),SqFtBuffere,Est_TotPop,Est_TotMinority,Est_TotPov,Est_TotLEP,Est_TotPop_Density,Transfer,CorrespondingStopID
0,783,0,4.034183e+06,1464,1254,697,37,7456.09,Yes,58.0
1,784,100,4.008440e+06,1481,1270,730,37,7542.67,No,NaN
2,785,200,3.983578e+06,1498,1287,762,37,7629.25,No,NaN
3,787,400,3.934566e+06,1506,1295,804,38,7670.00,No,NaN
4,788,500,3.911622e+06,1496,1283,812,38,7619.07,Yes,675.0


In [5]:
transfer_distances = r25[r25['Transfer'] == 'Yes']['Distance from initial stop (feet)'].values
transfer_distances = transfer_distances.reshape((1, transfer_distances.shape[0]))
transfer_distances.shape

(1, 8)

In [6]:
non_transfer_distances = r25[r25['Transfer'] == 'No']['Distance from initial stop (feet)'].values
non_transfer_distances = non_transfer_distances.reshape((non_transfer_distances.shape[0], 1))
non_transfer_distances.shape

(562, 1)

In [7]:
transfer_matrix = np.repeat(transfer_distances, non_transfer_distances.shape[0], axis=0)
transfer_matrix.shape

(562, 8)

In [8]:
non_transfer_matrix = np.repeat(non_transfer_distances, transfer_distances.shape[1], axis=1)
non_transfer_matrix.shape

(562, 8)

In [9]:
distances_from_transfers = np.absolute(transfer_matrix - non_transfer_matrix)

In [10]:
fivehundred = 500 * np.ones(distances_from_transfers.shape)
less_than_500 = np.less(distances_from_transfers, fivehundred)

In [11]:
less_than_500

array([[ True,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       [ True,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True]])

In [12]:
stop_invalid = np.any(less_than_500, axis=1)
stop_invalid.shape

(562,)

In [13]:
stop_valid = np.logical_not(stop_invalid)
stop_valid.shape

(562,)

In [14]:
np.sum(stop_valid)

531

In [34]:
r25_valid_nontransfer = r25[r25['Transfer'] == 'No'][stop_valid]
r25_transfer = r25[r25['Transfer'] == 'Yes']
r25_valid = pd.concat([r25_valid_nontransfer, r25_transfer])
r25_valid.sort_values('Distance from initial stop (feet)', inplace=True)

In [35]:
r25_valid.head()

,UniqueNum,Distance from initial stop (feet),SqFtBuffere,Est_TotPop,Est_TotMinority,Est_TotPov,Est_TotLEP,Est_TotPop_Density,Transfer,CorrespondingStopID
0,783,0,4.034183e+06,1464,1254,697,37,7456.09,Yes,58.0
4,788,500,3.911622e+06,1496,1283,812,38,7619.07,Yes,675.0
11,796,1300,3.130417e+06,1212,996,675,48,6172.67,Yes,907.0
16,808,2500,1.681497e+06,767,562,310,44,3906.30,Yes,1795.0
17,818,3500,2.183919e+04,612,448,217,25,3116.89,No,1796.0


In [36]:
r25_valid.to_csv('../data/route_25_valid.csv', index=False)